In [4]:
%matplotlib inline

In [5]:
import numpy as np
import matplotlib.pyplot as plt
from msslib.utils import *

import random

from pystruct.models import GridCRF
import pystruct.learners as ssvm
from pystruct.utils import SaveLogger

from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

import datetime
import random
import pickle

In [6]:
test_runs = ["A","B","C","D","E","F"]

In [7]:
unwanted_recto = ["VA223RN","VA327RN","VA001RN","VA002RN","VA006RN","VA007RN","VA008RN","VA009RN"]

def get_set_paths(directory):
    return sorted(list(filter(lambda x: "RN" in x and not any(s in x for s in unwanted_recto) , listpaths(directory))))

In [8]:
label_dir = "/home/finlay/HMT/data/VenetusA/1200Hx900W/15blocks/labels/"
data_dir = "/home/finlay/HMT/data/VenetusA/1200Hx900W/15blocks/data/"

label_paths = get_set_paths(label_dir)
data_paths = get_set_paths(data_dir)

path_pairs = list(zip(label_paths, data_paths))
print(len(path_pairs))

150


In [9]:
# I'm training LDA on this as well as the random test set to ensure I always get the same number of classes
lda_data_path = "/home/finlay/HMT/data/VenetusA/1200Hx900W/15blocks/data/VA013RN-0014.npy"
lda_label_path = "/home/finlay/HMT/data/VenetusA/1200Hx900W/15blocks/labels/VA013RN-0014.npy"
lda_X = flatten_vector_matrix(np.load(lda_data_path))
lda_y = np.squeeze(flatten_vector_matrix(np.load(lda_label_path)))

In [12]:
for letter in test_runs:
    split_at = 20
    random.shuffle(path_pairs)
    train = path_pairs[:split_at]
    test = path_pairs[split_at:]
    
    oot_path = os.path.join("/home/finlay/HMT/data/VenetusA/1200Hx900W/15blocks/LDA_Grid_CRF", letter)
    mkdir(oot_path)
 
    crf = GridCRF(neighborhood=4)
    clf = ssvm.OneSlackSSVM(model=crf,n_jobs=-1, C=100, inference_cache=100, tol=.1,
                           logger=SaveLogger(oot_path+'.pickle', save_every=100))
    
    lda = LinearDiscriminantAnalysis()
    print(datetime.datetime.now())
    print("Training LDA %s" %letter)
    lda.fit(lda_X, lda_y)
    for l, d in train:
        X = flatten_vector_matrix(np.load(d))
        y = np.squeeze(flatten_vector_matrix(np.load(l)))
        lda.fit(X, y)
  

    print("Training %s" %letter)
    train_X = []
    train_y = []
    for l, d in train:
        print("Training on %s" %l)
        pre_X = lda.transform(flatten_vector_matrix(np.load(d)))
        train_X.append(pre_X.reshape(80,60,4))
        train_y.append(np.squeeze(np.load(l)).astype('int'))
    print(np.array(train_X).shape)
    clf.fit(train_X, train_y)
    print(datetime.datetime.now())

    pickle.dump(clf, open(os.path.join(oot_path + "_clf.pkl"), "wb" ) )
    print("Testing %s" %letter)
    test_results = []
    for l, d in test:
        pre_X = lda.transform(flatten_vector_matrix(np.load(d)))
        X = pre_X.reshape((80,60, pre_X.shape[1]))
        y = np.squeeze(flatten_vector_matrix(np.load(l)))
        pred = np.asarray(clf.predict([X]))
        test_results.append({
            'name': only_basename(l),
            'pred': pred.reshape(80,60),
            })

    print("Saving %s" %letter)
    for result in test_results:
        path = format_path(oot_path, 'npy', result['name'])
        np.save(path, scale_matrix(result['pred'], 15))
    print(datetime.datetime.now())

2016-06-07 21:28:30.661550
Training LDA A


/usr/lib/python3.5/site-packages/sklearn/discriminant_analysis.py:387: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")
/usr/lib/python3.5/site-packages/sklearn/discriminant_analysis.py:453: UserWarning: The priors do not sum to 1. Renormalizing
  UserWarning)


Training A
Training on /home/finlay/HMT/data/VenetusA/1200Hx900W/15blocks/labels/VA149RN-0150.npy
Training on /home/finlay/HMT/data/VenetusA/1200Hx900W/15blocks/labels/VA195RN-0196.npy
Training on /home/finlay/HMT/data/VenetusA/1200Hx900W/15blocks/labels/VA147RN-0148.npy
Training on /home/finlay/HMT/data/VenetusA/1200Hx900W/15blocks/labels/VA116RN-0288.npy
Training on /home/finlay/HMT/data/VenetusA/1200Hx900W/15blocks/labels/VA240RN-0241.npy
Training on /home/finlay/HMT/data/VenetusA/1200Hx900W/15blocks/labels/VA262RN-0432.npy
Training on /home/finlay/HMT/data/VenetusA/1200Hx900W/15blocks/labels/VA141RN-0142.npy
Training on /home/finlay/HMT/data/VenetusA/1200Hx900W/15blocks/labels/VA093RN-0094.npy
Training on /home/finlay/HMT/data/VenetusA/1200Hx900W/15blocks/labels/VA274RN-0444.npy
Training on /home/finlay/HMT/data/VenetusA/1200Hx900W/15blocks/labels/VA104RN-0105.npy
Training on /home/finlay/HMT/data/VenetusA/1200Hx900W/15blocks/labels/VA182RN-0183.npy
Training on /home/finlay/HMT/dat

/usr/lib/python3.5/site-packages/pystruct/learners/one_slack_ssvm.py:409: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  self.w = np.zeros(self.model.size_joint_feature)
/usr/lib/python3.5/site-packages/pystruct/learners/one_slack_ssvm.py:415: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  constraints.append((np.zeros(self.model.size_joint_feature), 0))
/usr/lib/python3.5/site-packages/pystruct/models/base.py:34: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  joint_feature_ = np.zeros(self.size_joint_feature)
/usr/lib/python3.5/site-packages/pystruct/learners/one_slack_ssvm.py:323: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  joint_feature_acc = np.zeros(self.model.size_joint_feature)


2016-06-08 01:25:58.418147
Testing A
Saving A
2016-06-08 01:30:05.127658
2016-06-08 01:30:05.320311
Training LDA B
Training B
Training on /home/finlay/HMT/data/VenetusA/1200Hx900W/15blocks/labels/VA262RN-0432.npy
Training on /home/finlay/HMT/data/VenetusA/1200Hx900W/15blocks/labels/VA036RN-0037.npy
Training on /home/finlay/HMT/data/VenetusA/1200Hx900W/15blocks/labels/VA087RN-0259.npy
Training on /home/finlay/HMT/data/VenetusA/1200Hx900W/15blocks/labels/VA115RN-0116.npy
Training on /home/finlay/HMT/data/VenetusA/1200Hx900W/15blocks/labels/VA169RN-0340.npy
Training on /home/finlay/HMT/data/VenetusA/1200Hx900W/15blocks/labels/VA175RN-0176.npy
Training on /home/finlay/HMT/data/VenetusA/1200Hx900W/15blocks/labels/VA248RN-0249.npy
Training on /home/finlay/HMT/data/VenetusA/1200Hx900W/15blocks/labels/VA184RN-0355.npy
Training on /home/finlay/HMT/data/VenetusA/1200Hx900W/15blocks/labels/VA288RN-0458.npy
Training on /home/finlay/HMT/data/VenetusA/1200Hx900W/15blocks/labels/VA017RN-0018.npy
Trai

In [1]:
clf

NameError: name 'clf' is not defined

In [11]:
lda.classes_

array([ 0.,  1.,  2.,  3.,  4.])

In [15]:
y.reshape(60,45).unique


AttributeError: 'numpy.ndarray' object has no attribute 'unique'

In [11]:
900/15

60.0